In [1]:
# !pip install catboost -q

In [2]:
# pip install mlflow -q

In [3]:
# !pip install lightgbm

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction import DictVectorizer

import pickle
import mlflow
from mlflow.tracking import MlflowClient

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment('housing-price')

<Experiment: artifact_location='/home/azureuser/MLOps_zoomcamp/mlruns/1', creation_time=1691845828281, experiment_id='1', last_update_time=1691845828281, lifecycle_stage='active', name='housing-price', tags={}>

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
data_path = 'data/Housing_dataset_train.csv'

## Modelling

In [9]:
state_to_zone = {
    "Abia": "South-East",
    "Adamawa": "North-East",
    "Akwa Ibom": "South-South",
    "Anambra": "South-East",
    "Bauchi": "North-East",
    "Bayelsa": "South-South",
    "Benue": "North-Central",
    "Borno": "North-East",
    "Cross River": "South-South",
    "Delta": "South-South",
    "Ebonyi": "South-East",
    "Edo": "South-South",
    "Ekiti": "South-West",
    "Enugu": "South-East",
    "Gombe": "North-East",
    "Imo": "South-East",
    "Jigawa": "North-West",
    "Kaduna": "North-West",
    "Kano": "North-West",
    "Katsina": "North-West",
    "Kebbi": "North-West",
    "Kogi": "North-Central",
    "Kwara": "North-Central",
    "Lagos": "South-West",
    "Nasarawa": "North-Central",
    "Niger": "North-Central",
    "Ogun": "South-West",
    "Ondo": "South-West",
    "Osun": "South-West",
    "Oyo": "South-West",
    "Plateau": "North-Central",
    "Rivers": "South-South",
    "Sokoto": "North-West",
    "Taraba": "North-East",
    "Yobe": "North-East",
    "Zamfara": "North-West",
}

In [10]:
house_type_ranks = {
    'Cottage': 1,
    'Bungalow': 2,
    'Townhouse': 3,
    'Terrace duplex': 4,
    'Detached duplex': 5,
    'Semi-detached duplex': 6,
    'Flat': 7,
    'Penthouse': 8,
    'Apartment': 9,
    'Mansion': 10
}

In [11]:
def preprocess(data_path):
    data = pd.read_csv(data_path)

    print(data.columns.tolist())
    
    data['zone'] = data['loc'].map(state_to_zone)
    data['title'] = data['title'].map(house_type_ranks)

    category_frequencies = data['loc'].value_counts(normalize=True)
    loc_frequency_mapping = category_frequencies.to_dict()
    data['loc'] = data['loc'].map(loc_frequency_mapping)

    data['rooms'] = data['bathroom'] + data['bedroom']
    data['bathroom_ratio'] = data['bathroom']/(data['bathroom'] + data['bedroom'])

    data['zone'] = data['zone'].astype('category').cat.codes

    print("_____________________________________________________________________________")
    print(data.head())

    X = data.drop(columns=['price'], axis=0)
    y = data.price

    return X, y



In [12]:
X_, y_ = preprocess(data_path)

['ID', 'loc', 'title', 'bedroom', 'bathroom', 'parking_space', 'price']
_____________________________________________________________________________
      ID       loc  title  bedroom  bathroom  parking_space        price  \
0   3583  0.028309    6.0      2.0       2.0            1.0  1149999.565   
1   2748  0.028227    9.0      NaN       2.0            4.0  1672416.689   
2   9261  0.027570    NaN      7.0       5.0            NaN  3364799.814   
3   2224  0.029786    5.0      5.0       2.0            4.0  2410306.756   
4  10300  0.026340    4.0      NaN       5.0            6.0  2600700.898   

   zone  rooms  bathroom_ratio  
0     2    4.0        0.500000  
1     5    NaN             NaN  
2     5   12.0        0.416667  
3     3    7.0        0.285714  
4     0    NaN             NaN  


In [34]:
mlflow.lightgbm.autolog(disable=True)

In [35]:
with mlflow.start_run():

    params = {
        'max_depth': 10,
        'n_estimators': 2000,
        'learning_rate': 0.002712819361612371,
        'colsample_bytree': 0.9484547548287134,
        'subsample': 0.8490126211976283
        }

    mlflow.log_params(params)

    fold_pred = []
    splits = 2
    fold = KFold(n_splits=splits)

    for data_index, test_index in fold.split(X_, y_):
        X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
        y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

        model = LGBMRegressor(**params, objective='rmse')
        model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)])
        model_preds = model.predict(X_test)

        rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
        print(f'err: {rmse}')
        fold_pred.append(rmse)

    RMSE = np.mean(fold_pred)

    mlflow.log_param("splits", splits)
    mlflow.log_metric("rmse", RMSE)
    
    with open('models/lgb.bin', 'wb') as f:
        pickle.dump(model, f)

    mlflow.log_artifact(local_path="models/lgb.bin", artifact_path="models_pickle")
    mlflow.lightgbm.log_model(model, artifact_path="models_mlflow")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000912 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1425.058519


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
err: 529324.2298639654
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000752 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1423.011279
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves.

In [41]:
with mlflow.start_run():

    params = {
        'max_depth': 10,
        'n_estimators': 2000,
        'subsample': 0.84,
        'learning_rate': 0.01,
        'n_estimators' : 2000
        }

    mlflow.log_params(params)

    fold_pred_1 = []
    splits = 10
    fold = KFold(n_splits=splits)

    for data_index, test_index in fold.split(X_, y_):
        X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
        y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

        model = CatBoostRegressor(**params)
        model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)], verbose=0)
        model_preds = model.predict(X_test)

        rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
        print(f'err: {rmse}')
        fold_pred_1.append(rmse)

    RMSE = np.mean(fold_pred_1)

    mlflow.log_param("splits", splits)
    mlflow.log_metric("rmse", RMSE)
    
    with open('models/cat.bin', 'wb') as f:
        pickle.dump(model, f)

    mlflow.log_artifact(local_path="models/cat.bin", artifact_path="models_pickle")
    mlflow.catboost.log_model(model, artifact_path="models_mlflow")

err: 466743.0169847345
err: 524069.56855255977
err: 638941.7498937332
err: 470119.679511906
err: 476696.0579938225
err: 588413.1371192657
err: 552106.7342063364
err: 494535.4359102119
err: 513112.49038561864
err: 633301.7921100028


In [42]:
with mlflow.start_run():

    params = {
        'max_depth': 10,
        'n_estimators': 2000,
        'subsample': 0.84,
        'learning_rate': 0.01,
        'n_estimators' : 2000
        }

    mlflow.log_params(params)

    fold_pred_1 = []
    splits = 10
    fold = KFold(n_splits=splits)

    for data_index, test_index in fold.split(X_, y_):
        X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
        y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

        model = XGBRegressor(**params)
        model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)], verbose=0)
        model_preds = model.predict(X_test)

        rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
        print(f'err: {rmse}')
        fold_pred_1.append(rmse)

    RMSE = np.mean(fold_pred_1)

    mlflow.log_param("splits", splits)
    mlflow.log_metric("rmse", RMSE)
    
    with open('models/xgb.bin', 'wb') as f:
        pickle.dump(model, f)

    mlflow.log_artifact(local_path="models/xgb.bin", artifact_path="models_pickle")
    mlflow.xgboost.log_model(model, artifact_path="models_mlflow")

err: 491728.36126421625
err: 547610.9804050468
err: 653658.618074101
err: 496614.536246212
err: 482955.2736791648
err: 615508.374548473
err: 557145.6781747503
err: 516872.42737316753
err: 581392.2704359363
err: 649455.7368549954


## Model Registry

In [50]:
client.list_experiments()

AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

# Hyperparameter Tuning

In [ ]:
import optuna

In [ ]:
# def objective(trial):

#     max_depth = trial.suggest_int('rf_max_depth', 2, 32)
#     n_estimators = trial.suggest_int('n_estimators', 100, 4000)
#     learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
#     subsample = trial.suggest_float('subsample', 0, 1)

#     params = {
#         'max_depth':max_depth,
#         'colsample_bytree': colsample_bytree,
#         'learning_rate': learning_rate,
#         'n_estimators': n_estimators,
#         'subsample': subsample,
#     }

#     X_data, X_val, y_data, y_val = data_test_split(X, y, random_state=RANDOM_STATE)

#     LGB = CatBoostRegressor(**params)
#     LGB.fit(X_data, y_data)
#     y_pred = LGB.predict(X_val)

#     error = mean_squared_error(y_val, y_pred, squared=False)

#     return error  # An objective value linked with the Trial object.

#  # Invoke optimization of the objective function.

In [ ]:
def objective(trial):

    max_depth = trial.suggest_int('rf_max_depth', 2, 16)
    n_estimators = trial.suggest_int('n_estimators', 100, 4000)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
    subsample = trial.suggest_float('subsample', 0, 1)

    params = {
        'max_depth':max_depth,
#         'colsample_bytree': colsample_bytree,
        'learning_rate': learning_rate,
        'n_estimators': n_estimators,
        'subsample': subsample,
    }

    X_data, X_val, y_data, y_val = data_test_split(X, y, random_state=RANDOM_STATE)

    CAT = CatBoostRegressor(**params)
    CAT.fit(X_data, y_data, verbose=0)
    y_pred = CAT.predict(X_val)

    error = mean_squared_error(y_val, y_pred, squared=False)

    return error  # An objective value linked with the Trial object.

In [ ]:
study = optuna.create_study(direction='minimize')  # Create a new study.
study.optimize(objective, n_trials=100)

[I 2023-08-06 14:25:46,580] A new study created in memory with name: no-name-15fb2ab2-531f-4dff-b48b-b1376f15677d
[I 2023-08-06 14:25:53,716] Trial 0 finished with value: 576129.2543902358 and parameters: {'rf_max_depth': 11, 'n_estimators': 529, 'learning_rate': 0.06455367174443834, 'subsample': 0.43625841484720584}. Best is trial 0 with value: 576129.2543902358.
/home/gbotemi/.local/lib/python3.10/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
[I 2023-08-06 14:26:03,780] Trial 1 finished with value: 671184.3102130804 and parameters: {'rf_max_depth': 7, 'n_estimators': 3050, 'learning_rate': 0.000635824373713653, 'subsample': 0.5551330404486285}. Best is trial 0 with value: 576129.2543902358.
/home/gbotemi/.local/lib/python3.10/site-packages/catboost/core.py:1411: FutureWarning:

In [ ]:
study.best_trial

FrozenTrial(number=95, state=TrialState.COMPLETE, values=[551678.5994224877], datetime_start=datetime.datetime(2023, 8, 6, 14, 17, 1, 888573), datetime_complete=datetime.datetime(2023, 8, 6, 14, 17, 7, 916277), params={'rf_max_depth': 26, 'n_estimators': 3679, 'learning_rate': 0.002712819361612371, 'colsample_bytree': 0.9484547548287134, 'subsample': 0.8490126211976283}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'rf_max_depth': IntDistribution(high=32, log=False, low=2, step=1), 'n_estimators': IntDistribution(high=4000, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.0, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.0, step=None)}, trial_id=95, value=None)

In [ ]:
trial = study.best_trial

In [ ]:
trial.value

551678.5994224877

In [ ]:
trial.params

{'rf_max_depth': 26,
 'n_estimators': 3679,
 'learning_rate': 0.002712819361612371,
 'colsample_bytree': 0.9484547548287134,
 'subsample': 0.8490126211976283}